In [1]:
import sys
# from collections import Counter, OrderedDict
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab_edited import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# These will already be installed in the testing environment so disregard the
# amount of time (~1 minute) it takes to install.
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 123632 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u432-ga~us1-0ubuntu2~22.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u432-ga~us1-0ub

In [3]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [4]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
conf.set("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [5]:
spark

In [6]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
import signal

AUTH_TIMEOUT = 100

def handler(signum, frame):
  raise Exception("Authentication timeout!")

try:
  signal.signal(signal.SIGALRM, handler)
  signal.alarm(AUTH_TIMEOUT)
  auth.authenticate_user()
  signal.alarm(0)
except:
  pass

In [7]:
import os
from pathlib import Path
from google.colab import auth
from google.cloud import storage

# Alternative authentication method
!gcloud auth login

# Set your project and bucket info
project_id = 'hw2-project-444515'
data_bucket_name = '314616475'

# Configure project
!gcloud config set project {project_id}

# Create directory and copy file directly
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass
except:
    !mkdir -p wikidumps
    !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/"


You are running on a Google Compute Engine virtual machine.
It is recommended that you use service accounts for authentication.

You can run:

  $ gcloud config set account `ACCOUNT`

to switch accounts if necessary.

Your credentials may be visible to others with access to this
virtual machine. Are you sure you want to authenticate with
your personal account?

Do you want to continue (Y/n)?  y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.

In [8]:
import os
import shutil

!mkdir -p text_index
!mkdir -p title_index
!mkdir -p anchor_index

def move_bin_files(folder_name):
    """
    Moves all files ending with '.bin' in the current directory
    to the specified folder.

    Args:
        folder_name (str): The name of the destination folder.
    """
    for filename in os.listdir():
        if filename.endswith(".bin") or filename.endswith(".pkl"):
            source_path = os.path.abspath(filename)
            destination_path = os.path.join(folder_name, filename)

            try:
                shutil.move(source_path, destination_path)
            except Exception as e:
                print(f"Error moving '{filename}': {e}")


In [9]:
from pathlib import Path
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
parquetFile.show()

+---+--------------------+--------------------+--------------------+
| id|               title|                text|         anchor_text|
+---+--------------------+--------------------+--------------------+
| 12|           Anarchism|'''Anarchism''' i...|[{23040, politica...|
| 25|              Autism|'''Autism''' is a...|[{492271, Clinica...|
| 39|              Albedo|thumb|upright=1.3...|[{679294, diffuse...|
|290|                   A|'''A''', or '''a'...|[{290, See below}...|
|303|             Alabama|'''Alabama''' () ...|[{351590, Yellowh...|
|305|            Achilles|thumb|260px|Ancie...|[{1076007, potter...|
|307|     Abraham Lincoln|'''Abraham Lincol...|[{1827174, Alexan...|
|308|           Aristotle|'''Aristotle''' (...|[{1389981, bust},...|
|309|An American in Paris|'''''An American ...|[{13066, George G...|
|316|Academy Award for...|The '''Academy Aw...|[{39842, Academy ...|
|324|      Academy Awards|The '''Academy Aw...|[{649481, film in...|
|330|             Actrius|'''''Act

In [10]:
# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
doc_text_pairs = parquetFile.limit(1000).select("text", "id").rdd
doc_title_pairs = parquetFile.limit(1000).select("title", "id").rdd
doc_anchor_pairs = parquetFile.limit(1000).select("anchor_text", "id").rdd

In [11]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
all_stopwords = english_stopwords.union(corpus_stopwords)

def tokenize(text):
  english_stopwords = frozenset(stopwords.words('english'))
  corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
  RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

  all_stopwords = english_stopwords.union(corpus_stopwords)
  stemmer = PorterStemmer()
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  stemmed = [stemmer.stem(token) for token in tokens if token not in all_stopwords]
  return stemmed

def stemmed_word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = tokenize(text)
  token_counts = Counter(tokens)  # Calculate term frequencies efficiently
  res_list = [
        (token, (id, count))
        for token, count in token_counts.items()
        if token not in all_stopwords
    ]
  return res_list


def len_tokens(text, id):
  english_stopwords = frozenset(stopwords.words('english'))
  corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
  RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
  all_stopwords = english_stopwords.union(corpus_stopwords)

  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [x for x in tokens if x not in all_stopwords]
  return (id, len(tokens))

In [12]:
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  res_rdd = postings.map(lambda x: (x[0], len(x[1])))
  return res_rdd
  # raise NotImplementedError()

In [13]:
text_doc_lengths_dict = {}
title_doc_lengths_dict = {}


# Apply the function to your RDD
text_len_pairs = doc_text_pairs.map(lambda x: len_tokens(x[0], x[1]))
text_doc_lengths_dict = text_len_pairs.collectAsMap()

title_len_pairs = doc_title_pairs.map(lambda x: len_tokens(x[0], x[1]))
title_doc_lengths_dict = title_len_pairs.collectAsMap()

In [14]:
stemmed_text_word_counts = doc_text_pairs.flatMap(lambda x: stemmed_word_count(x[0], x[1]))
stemmed_title_word_counts = doc_title_pairs.flatMap(lambda x: stemmed_word_count(x[0], x[1]))

new_pairs = doc_anchor_pairs.flatMap(lambda pair: [(p[1], p[0]) for p in pair[0]])
stemmed_anchor_word_counts = new_pairs.flatMap(lambda x: stemmed_word_count(x[0], x[1]))

In [ ]:
# stemmed_word_count_res = stemmed_word_count(text, id)
# print("stemmed:", stemmed_word_count_res)
# assert stemmed_word_count_res[0][0] == 'test'
# assert len(stemmed_word_count_res)

In [16]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  # sorted_pl = sorted(unsorted_pl, key=lambda x: x[0])
  return sorted(unsorted_pl, key=lambda x: x[0])
  # raise NotImplementedError()

In [17]:
text_postings = stemmed_text_word_counts.groupByKey().mapValues(reduce_word_counts)
title_postings = stemmed_title_word_counts.groupByKey().mapValues(reduce_word_counts)
anchor_postings = stemmed_anchor_word_counts.groupByKey().mapValues(reduce_word_counts)

def new_count(posting):
    counts = Counter(p[0] for p in posting)
    return list(counts.items())

# Apply the group function to the values of each key in posting_anchor
anchor_postings = anchor_postings.map(lambda x: (x[0], new_count(x[1])))

In [18]:
text_postings_filtered = text_postings.filter(lambda x: len(x[1])>10)
title_postings_filtered = title_postings.filter(lambda x: len(x[1])>10)
anchor_postings_filtered = anchor_postings.filter(lambda x: len(x[1])>10)

In [19]:
text_w2df = calculate_df(text_postings_filtered)
title_w2df = calculate_df(title_postings_filtered)
anchor_w2df = calculate_df(anchor_postings_filtered)

text_w2df_dict = text_w2df.collectAsMap()
title_w2df_dict = title_w2df.collectAsMap()
anchor_w2df_dict = anchor_w2df.collectAsMap()

In [20]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings, base_dir):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''
  # YOUR CODE HERE
  bucket_rdd = postings.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey()

  def write_bucket(b_w_pl):
      bucket_id, word_posting_pairs = b_w_pl
      return InvertedIndex.write_a_posting_list((bucket_id, list(word_posting_pairs)), base_dir)

  posting_locs_list = bucket_rdd.map(write_bucket)

  return posting_locs_list

In [21]:
text_posting_locs_list = partition_postings_and_write(text_postings_filtered, 'text_index').collect()

text_super_posting_locs = defaultdict(list)
for posting_loc in text_posting_locs_list:
  for k, v in posting_loc.items():
    text_super_posting_locs[k].extend(v)

# Create inverted index instance
text_inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
text_inverted.posting_locs = text_super_posting_locs
# Add the token - df dictionary to the inverted index
text_inverted.df = text_w2df_dict
# write the global stats out
# text_inverted.write_index('.', 'index')
text_inverted.write_index('text_index', 'text')

In [22]:
title_posting_locs_list = partition_postings_and_write(title_postings_filtered, 'title_index').collect()

title_super_posting_locs = defaultdict(list)
for posting_loc in title_posting_locs_list:
  for k, v in posting_loc.items():
    title_super_posting_locs[k].extend(v)

# Create inverted index instance
title_inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
title_inverted.posting_locs = title_super_posting_locs
# Add the token - df dictionary to the inverted index
title_inverted.df = title_w2df_dict
# write the global stats out
title_inverted.write_index('title_index', 'title')

In [23]:
anchor_posting_locs_list = partition_postings_and_write(anchor_postings_filtered,'anchor_index').collect()

anchor_super_posting_locs = defaultdict(list)
for posting_loc in anchor_posting_locs_list:
  for k, v in posting_loc.items():
    anchor_super_posting_locs[k].extend(v)

# Create inverted index instance
anchor_inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
anchor_inverted.posting_locs = anchor_super_posting_locs
# Add the token - df dictionary to the inverted index
anchor_inverted.df = anchor_w2df_dict
# write the global stats out
anchor_inverted.write_index('anchor_index', 'anchor')

In [24]:
TUPLE_SIZE = 6
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [25]:
idx_text = InvertedIndex.read_index('text_index', 'text')
idx_title = InvertedIndex.read_index('title_index', 'title')
idx_anchor = InvertedIndex.read_index('anchor_index', 'anchor')

In [26]:
# Modify BM25 class to overcome DL - DL: fetches the document length of each document.

import math
from itertools import chain
import time
import builtins


# When preprocessing the data have a dictionary of document length for each document saved in a variable called `DL`.
class BM25_from_index:
    """
    Best Match 25.
    ----------
    k1 : float, default 1.5

    b : float, default 0.75

    index: inverted index
    """

    def __init__(self,index,DL,k1=1.5, b=0.75):
        self.b = b
        self.k1 = k1
        self.index = index
        self.DL = DL
        self.N = len(DL)
        # self.AVGDL = sum(DL.values())/self.N
        # self.AVGDL = sum(list(DL.values())) / self.N
        self.AVGDL = builtins.sum(DL.values()) / self.N
        self.words, self.pls = zip(*self.index.posting_lists_iter())

    def calc_idf(self,list_of_tokens):
        """
        This function calculate the idf values according to the BM25 idf formula for each term in the query.

        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']

        Returns:
        -----------
        idf: dictionary of idf scores. As follows:
                                                    key: term
                                                    value: bm25 idf score
        """
        idf = {}
        for term in list_of_tokens:
            if term in self.index.df.keys():
                n_ti = self.index.df[term]
                idf[term] = math.log(1 + (self.N - n_ti + 0.5) / (n_ti + 0.5))
            else:
                pass
        return idf


    def search(self, queries,N=3):
        """
        This function calculate the bm25 score for given query and document.
        We need to check only documents which are 'candidates' for a given query.
        This function return a dictionary of scores as the following:
                                                                    key: query_id
                                                                    value: a ranked list of pairs (doc_id, score) in the length of N.

        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']
        doc_id: integer, document id.

        Returns:
        -----------
        score: float, bm25 score.
        """
        # YOUR CODE HERE
        query_scores = {} # init dict

        for query_id, query_tokens in queries.items():
            self.idf = self.calc_idf(query_tokens) # calc idf for current query

            if not self.idf:
                query_scores[query_id] = []  # no relevant documents
                continue

            # getting candidtaes
            candidate_docs = set()
            for term in query_tokens:
                if term in self.index.df:
                    posting_list = self.pls[self.words.index(term)]
                    # print(f"posting_list for term {term} is:{posting_list}")
                    candidate_docs.update([doc_id for doc_id, _ in posting_list])
                    # print(f"candidate_docs for term {term} are: {candidate_docs}")

            # calc common terms' data
            common_terms_data = {}
            for term in query_tokens:
                if term in self.index.df:
                    posting_list = self.pls[self.words.index(term)]
                    common_terms_data[term] = dict(posting_list)

            # calc scores for candidates
            doc_scores = {}
            for doc_id in candidate_docs:
                score = 0.0
                doc_len = self.DL[doc_id]

                for term in query_tokens:
                    if term in common_terms_data and doc_id in common_terms_data[term]:
                        freq = common_terms_data[term][doc_id]
                        numerator = self.idf[term] * freq * (self.k1 + 1)
                        denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.AVGDL)
                        score += (numerator / denominator)

                doc_scores[doc_id] = score

            sorted_scores = sorted(doc_scores.items(), key=lambda item: item[1], reverse=True)
            top_n_docs = sorted_scores[:N]

            query_scores[query_id] = top_n_docs

        return query_scores

    def _score(self, query, doc_id):
        """
        This function calculate the bm25 score for given query and document.

        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']
        doc_id: integer, document id.

        Returns:
        -----------
        score: float, bm25 score.
        """
        score = 0.0
        doc_len = DL[str(doc_id)]

        for term in query:
            if term in self.index.term_total.keys():
                term_frequencies = dict(self.pls[self.words.index(term)])
                if doc_id in term_frequencies.keys():
                    freq = term_frequencies[doc_id]
                    numerator = self.idf[term] * freq * (self.k1 + 1)
                    denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.AVGDL)
                    score += (numerator / denominator)
        return score

In [27]:
bm25_text = BM25_from_index(idx_text, text_doc_lengths_dict)
bm25_title = BM25_from_index(idx_title, title_doc_lengths_dict)
bm25_anchor = BM25_from_index(idx_anchor, text_doc_lengths_dict)

In [28]:
print(bm25_text.index.posting_locs)
print(bm25_title.index.posting_locs)
print(bm25_anchor.index.posting_locs)

defaultdict(<class 'list'>, {'polit': [('text_index/text_index_74_000.bin', 0)], 'revolutionari': [('text_index/text_index_74_000.bin', 1902)], 'cynic': [('text_index/text_index_74_000.bin', 2466)], 'gave': [('text_index/text_index_74_000.bin', 2562)], 'fertil': [('text_index/text_index_74_000.bin', 4350)], 'lead': [('text_index/text_index_74_000.bin', 4740)], 'suppress': [('text_index/text_index_74_000.bin', 7434)], 'lenin': [('text_index/text_index_74_000.bin', 7974)], 'feminist': [('text_index/text_index_74_000.bin', 8088)], 'affin': [('text_index/text_index_74_000.bin', 8268)], 'primari': [('text_index/text_index_74_000.bin', 8496)], 'combin': [('text_index/text_index_74_000.bin', 10050)], 'tool': [('text_index/text_index_74_000.bin', 12204)], 'increasingli': [('text_index/text_index_74_000.bin', 12996)], 'grant': [('text_index/text_index_74_000.bin', 13824)], 'rais': [('text_index/text_index_74_000.bin', 15084)], 'lessen': [('text_index/text_index_74_000.bin', 16524)], 'outlin': [

In [35]:
print(read_posting_list(idx_text, 'alfonso'))
print(read_posting_list(idx_title, 'alfonso'))
print(read_posting_list(idx_anchor, 'alfonso'))

[(690, 1), (736, 1), (1129, 1), (1154, 1), (1247, 6), (1332, 1), (1360, 1), (1442, 1), (1491, 1), (1496, 1), (1499, 1), (1649, 4), (1657, 12), (1658, 3), (1659, 3), (1660, 5), (1663, 6), (1664, 11), (1676, 34), (1677, 62), (1679, 34), (1683, 2), (1684, 17), (1686, 41), (1688, 67), (1689, 8), (1691, 8), (1692, 5), (1694, 4), (1696, 3), (1862, 1), (1878, 1), (1879, 15), (1937, 1), (2194, 1), (2315, 1), (2329, 2), (2395, 1), (2417, 2)]
[(1247, 1), (1664, 1), (1676, 1), (1677, 1), (1679, 1), (1683, 1), (1684, 1), (1686, 1), (1689, 1), (1691, 1), (1692, 1), (1694, 1), (1696, 1), (1879, 1), (2329, 1)]
[(1247, 1), (1664, 3), (1676, 8), (1677, 10), (1679, 4), (1683, 2), (1684, 2), (1686, 3), (1688, 1), (1878, 1), (1879, 1), (66789, 4), (66796, 3), (69603, 4), (69607, 7), (70642, 5), (70643, 5), (70650, 9), (74249, 2), (74252, 2), (151250, 3), (151257, 2), (230254, 1), (475100, 1), (609471, 1), (704081, 1), (884434, 1), (994045, 1), (1228198, 1), (1321511, 2), (1400149, 1), (1520970, 2), (17449

In [32]:
print(bm25_text.words)
print(bm25_title.words)

('polit', 'revolutionari', 'cynic', 'gave', 'fertil', 'lead', 'suppress', 'lenin', 'feminist', 'affin', 'primari', 'combin', 'tool', 'increasingli', 'grant', 'rais', 'lessen', 'outlin', 'hugh', 'plant', '9th', 'planter', 'passag', 'influx', 'reptil', 'hall', 'visitor', 'championship', 'referendum', '102', 'shoal', '274', 'brightest', "plato'", 'heroic', 'choru', 'engrav', 'samuel', 'cousin', 'euclid', 'intact', 'deliber', 'uncertainti', 'mysteri', 'lighter', 'tenor', 'laura', 'pierc', '45th', 'reign', 'maltes', 'köppen', 'airplan', 'recurr', 'warlord', 'augustinian', 'kid', "that'", 'thrace', 'trunk', '1780', 'penn', 'henderson', 'adjoin', 'ventil', 'spiral', 'subsidi', 'berri', 'tranquil', 'co-author', 'neon', 'magician', 'thomson', '1649', 'ethel', 'sicilian', 'takeoff', '640', 'entourag', 'federico', '1549', '1619', 'vic', 'philosophi', 'contribut', 'add', 'deriv', 'sentiment', '1872', 'synthesi', 'soviet', 'type', 'concept', 'benjamin', 'give', 'video', 'illus', 'skill', 'syndrom',

In [ ]:
queries = {
    1: ['revolutionari'],
    2: ['lenin'],
    3: ['feminist']
}
bm25_queries_score_train_text = bm25_text.search(queries,N=10)
for score in bm25_queries_score_train_text.values():
    print(score)

[(12, 4.879642078087676), (771, 4.86235544904703), (1973, 4.786908409142804), (1363, 4.132222745765595), (1376, 3.993825111877486), (2360, 3.806082657045014), (620, 3.729513077090556), (2210, 3.6490814674429073), (1013, 3.591399025702808), (1448, 3.5747232610697206)]
[(1625, 6.448370113417525), (676, 5.791176697533045), (1020, 5.7085595934679265), (620, 5.543502724587618), (2447, 5.005026086005725), (1710, 4.068752002627364), (2439, 4.001997462981853), (1794, 3.477654390467359), (1592, 3.286663265246951), (1806, 3.2743610763677737)]
[(569, 6.795554353057614), (2126, 6.6408617559148455), (2314, 6.472896653519855), (1948, 6.251222194420574), (12, 6.153244151968032), (1653, 4.871846257022749), (2029, 4.807402990768633), (1416, 4.479609998399323), (1384, 4.31006278989696), (864, 4.102338870694428)]


In [ ]:
print(text_doc_lengths_dict.get(1028))

4863
